In [3]:
import torch

embedding = torch.rand(10, 56,100,100)

In [14]:
# let's get the client and server model from the embedding
from Models.mobilenetv2 import MobileNetV2
from Dataloaders.dataloader_cifar10 import Dataloader_cifar10
import torch.nn as nn
import torch.optim as optim
import torch
from tqdm import tqdm
import os
import sys
import cv2

model = MobileNetV2(num_classes=10)
client_model = model.get_client_model()
server_model = model.get_server_model()

# split the server model into 2 parts
server_model_1 = server_model[:len(server_model)-1]
server_model_2 = server_model[-1]

# client model and server 1 are fixed, server 2 is trainable
client_model = client_model.train()
server_model_1 = server_model_1.train()
server_model_2 = server_model_2.train()

loss = torch.nn.CrossEntropyLoss()
# https://github.com/d-li14/mobilenetv2.pytorch
# batch size 256; epoch 150; learning rate 0.05; LR decay strategy cosine; weight decay 0.00004
optimizer = torch.optim.Adam(server_model_2.parameters(), lr=0.05)

# put them in cuda
client_model = client_model.cuda()
server_model_1 = server_model_1.cuda()
server_model_2 = server_model_2.cuda()

# get the data
train, test, val, labels = Dataloader_cifar10(128, 2014)

# train
epoch = 20
for i in tqdm(range(epoch)):
    for data in train:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        # with torch.no_grad():
        outputs = client_model(inputs)
        outputs = server_model_1(outputs)
        outputs = server_model_2(outputs)
        # get the index of the max value of the output?
        loss_train = loss(outputs, labels)
        loss_train.backward()
        optimizer.step()
    print('train loss: ',loss_train.item())
    
    # val
    if i % 5 == 0:
        for data in val:
            inputs, labels = data
            inputs = inputs.cuda()
            labels = labels.cuda()
            with torch.no_grad():
                outputs = client_model(inputs)
                outputs = server_model_1(outputs)
                outputs = server_model_2(outputs)
            loss_val = loss(outputs, labels)
        print('val loss: ', loss_val.item())

# test
correct = 0
total = 0
with torch.no_grad():
    for data in test:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = client_model(inputs)
        outputs = server_model_1(outputs)
        outputs = server_model_2(outputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total)) 


 

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/100 [00:00<?, ?it/s]

train loss:  0.6799212098121643


  1%|          | 1/100 [00:13<21:46, 13.20s/it]

val loss:  1.2833060026168823


...

 99%|█████████▉| 99/100 [17:25<00:10, 10.19s/it]

train loss:  1.7990056276321411


100%|██████████| 100/100 [17:35<00:00, 10.55s/it]

train loss:  2.167130470275879


Accuracy of the network on the 10000 test images: 79 %
